In [1]:
from meb import utils
from meb import datasets

import numpy as np
import pandas as pd
from meb.utils.utils import dataset_aus
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.base import BaseEstimator


pd.set_option("display.max_columns", 50)
%load_ext autoreload
%autoreload 2

In [2]:
c = datasets.CrossDataset(ignore_validation=True)
df = c.data_frame

In [3]:
def predict_emotions_from_aus(
    df: pd.DataFrame,
    dataset_name: str,
    model: BaseEstimator,
    emotion_column_name: str = "emotion",
    action_units: list = []
):
    action_units = action_units if action_units else df.loc[:, "AU1":].columns
    df_dataset = df[df["dataset"] == dataset_name]
    
    aus = np.array(df_dataset.loc[:, action_units])
    emotions = df_dataset[emotion_column_name].astype("str")
    model.fit(aus, emotions)
    predicted_emotions = model.predict(aus)
    f1 = f1_score(emotions, predicted_emotions, average="macro")
    print(f1 * 100)
    return emotions, predicted_emotions

## Results from datasets

4DME shows result from objective and CASME3 from self-reported

In [4]:
dataset_names = ["casme", "casme2", "samm", "mmew", "casme3a"]
for dataset_name in dataset_names:
    print(dataset_name)
    _, _ = predict_emotions_from_aus(df, dataset_name, GradientBoostingClassifier())

casme
82.61934608088903
casme2
87.20575423499223
samm
96.07897007086092
mmew
95.82190502610854
casme3a
32.36310309631205


## 4DME Inclusion of (k) AUs

In [5]:
fourd_df = datasets.Fourd(ignore_ks=True).data_frame
fourd_df.insert(3, "dataset", "fourd")

In [6]:
_, _ = predict_emotions_from_aus(fourd_df, "fourd", GradientBoostingClassifier())

100.0


## 4DME self-reported and video emotions

In [7]:
fd_extra = "/research/cmv/development/Databases/micro_expressions/4DMicro/Original data/4DME_Labelling_Micro_extras_release v1.xlsx"
sdf = pd.read_excel(fd_extra)
sdf = sdf.rename(dict(sub_ID="subject"), axis=1)
sdf = sdf.drop(["fold", "subject", "fold_name"], axis=1)
sdf = pd.concat([sdf, df[df["dataset"] == "fourd"].reset_index()], axis=1)

In [8]:
_, _ = predict_emotions_from_aus(sdf, "fourd", GradientBoostingClassifier(), "self-report emo'")

34.85974484801464


In [9]:
_, _ = predict_emotions_from_aus(sdf, "fourd", GradientBoostingClassifier(), "video emo'")

49.98483044547726


## CASME3 Objective

In [10]:
c3_extra = "/research/cmv/development/Databases/micro_expressions/CASME3/Original data/part_A/annotation/CAS(ME)3_part_A_v2.xls"
odf = pd.read_excel(c3_extra)
odf = odf[odf["Expression type"] == "Micro-expression"].reset_index()
sdf = df[df["dataset"] == "casme3a"].reset_index()
sdf.insert(8, "Objective class", odf["Objective class"])

In [11]:
_, _ = predict_emotions_from_aus(sdf, "casme3a", GradientBoostingClassifier(), "Objective class")

100.0


# Clashes

In [12]:
def au_clashes(df: pd.DataFrame, dataset_name: str = None) -> dict:
    """
    Sees if the AUs are the same but the emotion is different in a dataframe
    """
    # Remove any extras from AUs, e.g., R, (k), A
    dft = df.copy()
    dft["CAU"] = dft.loc[:, "AU1":].transpose().apply(lambda x: "+".join([au for au, idx in x.items() if idx]))
    # Create a dict with the au and emotion mappings
    d = {}
    dft = dft[dft["dataset"] == dataset_name] if dataset_name in dft["dataset"].unique() else dft
    for i, row in dft.iterrows():
        if row["CAU"] not in d:
            d.setdefault(row["CAU"], [])
        if row["emotion"] not in d[row["CAU"]]:
            d[row["CAU"]].append(row["emotion"])

    # See if there are clashes
    clashes = {}
    for s in d:
        if len(d[s]) > 1:
            clashes[s] = d[s]
    return clashes

## CASME2 Clashes

In [13]:
au_clashes(df, "casme2")

{'AU12': ['happiness', 'others'],
 'AU4': ['others', 'disgust'],
 'AU15': ['repression', 'happiness'],
 'AU12+AU15': ['repression', 'happiness'],
 'AU14': ['repression', 'others', 'happiness'],
 'AU17': ['others', 'repression'],
 'AU1': ['surprise', 'sadness'],
 'AU4+AU7': ['disgust', 'others'],
 'AU2+AU15': ['others', 'disgust'],
 'AU14+AU15': ['happiness', 'repression'],
 'AU4+AU12': ['happiness', 'others'],
 'AU6+AU7+AU12': ['others', 'happiness']}

#### CASME2 Clahes of AU14

In [14]:
df[(df["dataset"] == "casme2") & (df["AU"] == "14")]

,subject,material,onset,apex,offset,apexf,AU,emotion,n_frames,dataset,AU1,AU2,AU4,AU5,AU6,AU7,AU8,AU9,AU10,AU11,AU12,AU13,AU14,AU15,AU16,AU17,AU18,AU20,AU21,AU22,AU23,AU24,AU25,AU26,AU28,AU30,AU31,AU34,AU38,AU39,AU43,AU45,AU56,AU61,AU63,AU64
201,02,EP06_01f,56,121,155,65,14,repression,100,casme2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
202,02,EP06_02f,36,64,111,28,14,repression,76,casme2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
203,02,EP08_04,16,69,106,53,14,others,91,casme2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
224,05,EP03_01,41,68,101,27,14,happiness,61,casme2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
299,12,EP03_02,26,59,91,33,14,others,66,casme2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
310,13,EP01_02,76,101,116,25,14,others,41,casme2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
311,13,EP02_02,11,34,68,23,14,others,58,casme2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
312,13,EP02_03,26,59,66,33,14,others,41,casme2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
317,14,EP04_04f,11,44,86,33,14,others,76,casme2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
318,14,EP09_03,6,29,41,23,14,happiness,36,casme2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## CASME3 Clashes

In [15]:
au_clashes(df, "casme3a")

{'AU5': ['others', 'surprise', 'fear'],
 'AU4': ['disgust', 'others', 'anger', 'surprise', 'sad', 'happy', 'fear'],
 'AU12': ['others', 'anger', 'surprise', 'disgust', 'fear', 'happy'],
 'AU38': ['surprise', 'happy', 'disgust', 'anger', 'others', 'fear', 'sad'],
 'AU24': ['happy', 'others', 'sad', 'disgust', 'surprise', 'fear'],
 'AU25+AU26': ['surprise', 'others'],
 'AU16+AU24': ['disgust', 'others', 'fear'],
 'AU18': ['fear', 'others', 'surprise', 'happy', 'disgust'],
 'AU28': ['fear', 'sad', 'others'],
 'AU14': ['anger',
  'disgust',
  'others',
  'sad',
  'happy',
  'fear',
  'surprise',
  'Others'],
 'AU15': ['happy', 'disgust', 'fear'],
 'AU10': ['others', 'anger', 'disgust', 'happy'],
 'AU2': ['others', 'surprise', 'fear'],
 'AU25': ['others', 'surprise', 'disgust'],
 'AU14+AU24': ['fear', 'sad', 'disgust', 'anger', 'others'],
 'AU1+AU2': ['others', 'surprise', 'disgust', 'fear', 'sad'],
 'AU16': ['others', 'surprise'],
 'AU7': ['others', 'disgust', 'anger', 'surprise', 'sad', '

#### CASME3 Clahes of AU4

In [16]:
df[(df["dataset"] == "casme3a") & (df["AU"] == "4")]

,subject,material,onset,apex,offset,apexf,AU,emotion,n_frames,dataset,AU1,AU2,AU4,AU5,AU6,AU7,AU8,AU9,AU10,AU11,AU12,AU13,AU14,AU15,AU16,AU17,AU18,AU20,AU21,AU22,AU23,AU24,AU25,AU26,AU28,AU30,AU31,AU34,AU38,AU39,AU43,AU45,AU56,AU61,AU63,AU64
1172,spNO.1,b,166,175,179,9,4,disgust,14,casme3a,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1176,spNO.1,f2,1568,1572,1581,4,4,others,14,casme3a,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1186,spNO.10,i,2030,2035,2098,5,4,disgust,69,casme3a,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1194,spNO.10,l2,1086,1090,1094,4,4,anger,9,casme3a,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1195,spNO.10,l3,1407,1414,1424,7,4,surprise,18,casme3a,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,spNO.77,m,250,257,275,7,4,disgust,26,casme3a,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017,spNO.8,e3,295,301,308,6,4,disgust,14,casme3a,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2023,spNO.8,k,1043,1047,1099,4,4,happy,57,casme3a,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2028,spNO.9,g,234,237,251,3,4,sad,18,casme3a,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
